# Projeto Buscas de Melhoria Iterativa e Satisfação de Restrições

## 1. Autores:

* Arthur Fernandes de Morais
* João Custódio de Faria Filho
* Raphael de Vasconcelos Nascimento

## 2. Objetivo do Trabalho e descrição da implementação:

O presente trabalho visa a resolver os problemas 1 e 2 descritos abaixo. Para a solução de ambos os problemas foi utilizado a linguagem `Python 3.7` e  implementado via `Juyter Notebooks`. Utilizou-se também, para sincronizar as modificações pelos autores, a plataforma do `github` por meio de um repositório contendo tanto os notebooks quanto os arquivos auxiliares à solução dos problemas que são:

* Para o problema 1: Notebook `q1.ipynb`
* Para o problema 2: Notebook `q2.ipynb`

## 3. Resultados obtidos

#### 2.1 Descrição da solução do seguinte problema:
Crie um sistema capaz de resolver o problema das N-Rainhas através de busca
de melhoria iterativa (hill climbing ou têmpera simulada) para N com os seguintes
valores 10, 20 e 25. Tabele os tempos de processamento para obter uma solução.

In [700]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [701]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import re
import random
import time

**Para o problema em questão desenvolvemos as seguintes funções:**

`score_state(state)`: Calcula o score de um dado estado com base no número de rainhas que se atacam. *Score = 1/(N rainhas + 1)*

`fitness_function(population)`: Calcula a probabilidade de se escolher dados cromossomo com base no seu score.

`selection(population, p)`: Retorna a subpopulação de cromossomos com base na sua probabilidade de ser escolhido.

`cross_over(population)`: Retorna uma nova população de indivíduos após serem cruzados e "misturados" seus cromossomos

`mutation(population, p)`: Adiciona mutação em um gene numa fração p de indivíduos da população.

`print_game(cromo)`: Apresenta a representação de um dado estado. Uma rainha no tabuleiro é representada por um *0* e um espaço vazio por um *1*

### Funções auxiliares do algorítmo genético:

In [763]:
def score_state(state):
    L = len(state)
    max_atacks = L*(L-1)/2
    state = np.insert(state, 0, 0, axis=0)
    score = 0
    for i in range(1, L+1):
        if i <= L:
            for j in range(i+1, L+1):
                if state[j] == state[i]:
                    score += 1
                elif (i - state[i]) == (j - state[j]):
                    score += 1
                elif state[i] + i == state[j] + j:
                    score += 1
    
    return 1/(score + 1)
#     return max_atacks - score -> Uma métrica que tabém poderia ser utilizada para balizar a qualidade do estado

def fitness_function(population):
    total_score = 0
    L = len(population)
    for person in population:
        total_score += score_state(person)
    
    if total_score == 0:
        return np.random.dirichlet(np.ones(L),size=1)[0]
    
    result = np.array([])
    for i in range(L):
        result = np.append(result, score_state(population[i])/total_score)
    
    return result

def selection(population, p):
    # Seleção usando o método da roleta. De toda população selecionar uma fração p
    L = len(population)
    
    n = 0
    if int(len(population)*p) % 2 != 0:
        n = int(len(population)*p) - 1
    else:
        n = int(len(population)*p)
            
    selected = np.random.choice(np.arange(0, L, 1), n, p=fitness_function(population), replace=False)
        
    
    result = [population[i] for i in selected]
            
    return result

def cross_over(population):
    selected = np.zeros(len(population))
    new_population = []
    for i in range(len(selected) - 1):
        if selected[i] != 1:
            #Seleciona esse indivíduo
            selected[i] = 1

            #Seleciona outro, à frente no vetor, aleatoriamente
            j = np.random.choice(len(selected))
            while ~(selected[j] == 0 or j <= i):
                j = np.random.choice(len(selected))

            selected[j] = 1
            
            p1 = population[i]
            p2 = population[j]
    
            #Seleciona ao acaso uma posição para o cross
            x = np.random.choice(len(population[0])) + 1

            tmp = p2[:x].copy()
            p2[:x], p1[:x]  = p1[:x], tmp

            new_population.append(p1)
            new_population.append(p2)
        
    
    return new_population

def mutation(population, p):
    L = int(len(population)*p)
    #Uma fração de p indivíduos irá sofrer uma mutação em um gene
    for i in range(L):
        #Seleciona ao acaso um indivíduo pra sofrer mutação
        idx = np.random.randint(population.shape[0], size=1)
        cromo = population[idx, :][0]
        
        #Seleciona ao acaso uma posição desse indivíduo
        posic = np.random.choice(len(cromo), 1)
        
        #Seleciona ao acaso um novo gene
        cromo[posic] = np.random.choice(len(cromo), 1) + 1
        
        #Substitui na população
        population[idx] = cromo
        
        
    return population

def print_game(cromo):
    L = len(cromo)
    
    result = np.full((L,L), 1)
    
    for i in range(L):
        result[cromo[i]-1][i] = 0
    
    print(result)

### Código do algorítmo genético:

No código propriamente dito do algorítmo genético abaixo foi considerado como critério de parada uma população de 100 vezes a população inicial. Além disso vale destacar que o fator *p* além de ser o fator de reprodução também foi escolhido como fator de mutação, ou seja p% de indivíduos da nova população, após o cruzamento sofrerá a mutação de um gene.

In [717]:
# Enquanto critério de parada não atingido
def evolution(population, p):
    start = time.perf_counter()
    L = len(population[0])
    # Calcular aptidão
    steps = 0
    while True:
        steps += 1
        score_aux = 0
        for cromo in population:
            if score_state(cromo) > score_aux:
                score_aux = score_state(cromo)
                cromo_aux = cromo
        
        if len(population) > 100*L:
            finish = time.perf_counter()
            print(f'Finished in {round(finish - start, 3)} second(s)')
            return cromo_aux, steps, (1/score_state(cromo_aux)) - 1, len(population)
        
        #Selecionar uma fração p de membros para reprodução
        sub_p = selection(population, p)
        
        #Aplicar cruzamento e adicionar filhos à população
        sub_p = cross_over(selection(population, p))
        for child in sub_p:
            population = np.vstack([population, child])
        
        #Realizar mutação em membros da população
        population = mutation(population, p)
        

Cada uma das saídas da função evolution contem o array do resultado do algorítmo, quantas gerações foram geradas, quantos pares de rainhas esse estado possui se atacando, e qual o tamanho da população final. Vale ressaltar que para os casos em questão foi escolhido uma taxa fixa de 80% de cruzamento ao longo das gerações e que a saída da função evolution também apresenta o tempo transcorrido para apresnetar a solução.

In [764]:
population = np.random.randint(1, 10, size=(4, 10))
population

array([[1, 3, 9, 9, 6, 3, 5, 5, 5, 6],
       [3, 5, 1, 1, 9, 8, 1, 2, 7, 1],
       [3, 4, 2, 1, 4, 1, 8, 5, 2, 7],
       [8, 3, 6, 4, 6, 5, 4, 3, 9, 8]])

In [765]:
cromo, steps, attacks, population = evolution(population, 0.8)

Finished in 0.679 second(s)


In [766]:
cromo

array([8, 3, 9, 4, 8, 1, 5, 2, 2, 6])

In [767]:
print_game(cromo)

[[1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 1 0 0 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 0 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


In [768]:
population = np.random.randint(1, 20, size=(4, 20))
population

array([[12,  4, 17,  7,  7, 14,  9, 11,  4, 10, 19, 17, 12,  9,  3,  9,
         9,  9,  6,  3],
       [18,  1,  1, 16,  6, 16,  4, 17, 13, 16,  6, 11, 10,  3,  5, 16,
        12,  5, 12, 10],
       [11,  2, 11, 11,  6, 11,  7, 17, 14, 13, 11,  7, 19, 18, 17, 15,
        10, 13, 16, 12],
       [18, 16, 13,  5,  2,  6, 13,  6,  3, 17,  6, 10,  8, 12, 10, 18,
        11, 15, 11, 10]])

In [769]:
cromo, steps, attacks, population = evolution(population, 0.8)

Finished in 7.033 second(s)


In [770]:
cromo

array([18, 15, 13, 20,  6, 14,  7,  2,  4,  2, 19, 17, 19,  1,  3,  9,  9,
        9, 18,  3])

In [771]:
print_game(cromo)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [772]:
population = np.random.randint(1, 25, size=(4, 25))
population

array([[17,  1, 12,  7,  9, 13,  5, 14,  1, 13,  5,  4, 10,  8,  7,  7,
         5,  7,  4,  1, 23, 23, 12, 15, 17],
       [ 4, 16,  4, 20, 16,  2,  5, 24,  3, 13,  4,  8,  2,  2, 20, 23,
        16,  2,  2, 17,  3,  9, 10, 11, 24],
       [19, 13, 11, 21, 11, 16, 24,  5, 14,  9, 16, 23,  4, 14, 24,  2,
        22, 19,  9,  2,  4, 19,  5,  3, 10],
       [12, 13, 14, 15, 14,  4, 23, 24,  8,  4,  7, 10, 24, 24,  9, 18,
         6, 18,  9, 12,  2,  4,  1, 21, 24]])

In [773]:
cromo, steps, attacks, population = evolution(population, 0.8)

Finished in 11.812 second(s)


In [774]:
cromo

array([15, 22, 11,  7,  9, 13,  5, 25,  1, 20, 23,  8,  2, 14, 25,  7, 10,
       19,  4, 17,  2,  9, 12, 20,  4])

In [775]:
print_game(cromo)

[[1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0]
 [1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 